# Image summary and visual question answering

This notebooks shows how to generate image captions and use the visual question answering with [LAVIS](https://github.com/salesforce/LAVIS). 

The first cell is only run on google colab and installs the [ammico](https://github.com/ssciwr/AMMICO) package.

After that, we can import `ammico` and read in the files given a folder path.

In [1]:
# if running on google colab
# flake8-noqa-cell
import os

if "google.colab" in str(get_ipython()):
    # update python version
    # install setuptools
    # %pip install setuptools==61 -qqq
    # install ammico
    %pip install git+https://github.com/ssciwr/ammico.git -qqq
    # mount google drive for data and API key
    from google.colab import drive

    drive.mount("/content/drive")

In [2]:
import ammico
from ammico import utils as mutils
from ammico import display as mdisplay
import ammico.summary as sm

In [3]:
# Here you need to provide the path to your google drive folder
# or local folder containing the images
images = mutils.find_files(
    path="data/",
    limit=10,
)

In [4]:
mydict = mutils.initialize_dict(images)

## Create captions for images and directly write to csv

Here you can choose between two models: "base" or "large". This will generate the caption for each image and directly put the results in a dataframe. This dataframe can be exported as a csv file.

The results are written into the columns `const_image_summary` - this will always be the same result (as always the same seed will be used). The column `3_non-deterministic summary` displays three different answers generated with different seeds, these are most likely different when you run the analysis again.

In [5]:
obj = sm.SummaryDetector(mydict)
summary_model, summary_vis_processors = obj.load_model(model_type="base")
# summary_model, summary_vis_processors = mutils.load_model("large")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 46e7044b-81c7-40dd-96d4-f00f47977854)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 60a0dbec-fe0d-4c08-ab66-81694c369895)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: dc7187e1-ebbb-4567-af33-a6895b248a79)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 59a281a5-bebf-4eda-9a26-a35771174657)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json


OSError: Can't load tokenizer for 'bert-base-uncased'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'bert-base-uncased' is the correct path to a directory containing all relevant files for a BertTokenizer tokenizer.

In [6]:
for key in mydict:
    mydict[key] = sm.SummaryDetector(mydict[key]).analyse_image(
        summary_model=summary_model, summary_vis_processors=summary_vis_processors
    )

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 24f208f6-43d7-47b6-8107-73577c6b2438)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: fd6ca06d-4f6b-47bb-8cf4-948c3ca85a08)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 8c8fd2ef-98fd-4aa6-8695-7ce208b547de)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 03ed1cf4-5e3e-4f84-a85d-019584a19d86)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json


OSError: Can't load tokenizer for 'bert-base-uncased'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'bert-base-uncased' is the correct path to a directory containing all relevant files for a BertTokenizer tokenizer.

Convert the dictionary of dictionarys into a dictionary with lists:

In [7]:
outdict = mutils.append_data_to_dict(mydict)
df = mutils.dump_df(outdict)

Check the dataframe:

In [8]:
df.head(10)

,filename
0,data/102730_eng.png
1,data/102141_2_eng.png
2,data/106349S_por.png


Write the csv file:

In [9]:
df.to_csv("data_out.csv")

## Manually inspect the summaries

To check the analysis, you can inspect the analyzed elements here. Loading the results takes a moment, so please be patient. If you are sure of what you are doing.

`const_image_summary` - the permanent summarys, which does not change from run to run (analyse_image).

`3_non-deterministic summary` - 3 different summarys examples that change from run to run (analyse_image). 

In [10]:
analysis_explorer = mdisplay.AnalysisExplorer(mydict, identify="summary")
analysis_explorer.run_server(port=8055)

TypeError: __init__() got an unexpected keyword argument 'identify'

## Generate answers to free-form questions about images written in natural language. 

Set the list of questions as a list of strings:

In [11]:
list_of_questions = [
    "How many persons on the picture?",
    "Are there any politicians in the picture?",
    "Does the picture show something from medicine?",
]

Explore the analysis using the interface:

In [12]:
analysis_explorer = mdisplay.AnalysisExplorer(mydict, identify="summary")
analysis_explorer.run_server(port=8055)

TypeError: __init__() got an unexpected keyword argument 'identify'

## Or directly analyze for further processing
Instead of inspecting each of the images, you can also directly carry out the analysis and export the result into a csv. This may take a while depending on how many images you have loaded.

In [13]:
for key in mydict:
    mydict[key] = sm.SummaryDetector(mydict[key]).analyse_questions(list_of_questions)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: fea2d868-d5bc-417d-9e00-4db1e10f2a39)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: f0ccba03-f0e5-43ae-be4b-91ed53e31894)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: b28cdc86-997c-466d-ad54-a6022a60c715)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 115b5242-dca3-428c-a002-00eadbc77c47)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json


OSError: Can't load tokenizer for 'bert-base-uncased'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'bert-base-uncased' is the correct path to a directory containing all relevant files for a BertTokenizer tokenizer.

## Convert to dataframe and write csv
These steps are required to convert the dictionary of dictionarys into a dictionary with lists, that can be converted into a pandas dataframe and exported to a csv file.

In [14]:
outdict2 = mutils.append_data_to_dict(mydict)
df2 = mutils.dump_df(outdict2)

In [15]:
df2.head(10)

,filename
0,data/102730_eng.png
1,data/102141_2_eng.png
2,data/106349S_por.png


In [16]:
df2.to_csv("data_out2.csv")